<a href="https://colab.research.google.com/github/arthur422tp/arthur422/blob/master/%E8%AE%93AI%E8%AE%80%E5%9C%8B%E4%B8%AD%E6%9C%83%E8%80%83%E9%96%B1%E8%AE%80%E6%B8%AC%E9%A9%97%E9%A1%8C%E7%9B%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#問題意識：如何讓AI可以寫國文閱讀測驗題目
#####方法:目前我們預計使用google的bert模型來進行預測，但由於bert的訓練方式是克漏字選填與預測下一句話，與閱讀測驗不太相似。所以方法一我們打算把閱讀測驗分成一句一句後，丟進bert做鑲嵌訓練，得到它的詞向量後。再把問題與選項連在一起，也丟給bert做鑲嵌訓練(每個選項分開做一次)，也取他的詞向量。對於文章與ABCD的詞向量進行餘弦相似度測驗後，看與文章中每句的相似度，【選出每個選項最高的分數，比較後最高者為答案】




##分工:
###107101064應數二 余嘉恆—資料收集、資料整理、資料清洗
###107204050呂修齊—資料收集、方法二網絡
###107204002陳懷沁、107204025陳嘉緯—資料收集、方法一網絡

題目的資料來源為：
1.教育部全國中小學題庫網
2.教育部所公布之歷屆會考題
###(均已確認智慧財產權，只要沒用來營運就可以)

資料收集與整理方面，收集題庫網內從國一至國三的題目，以工人智慧的方式純手工打造(key到excel上)，選擇出閱讀測驗題目後去除空格、轉為csv檔後，再丟入下述的程式轉換為bert所需格式([SEP]and[CLS])，之後再將所欲使用之資料轉換為向量，即可輸入程式

###例題（短文）：
#####「如果每個人都是特別的，那就沒有一個特別；如果每個人都得到一個獎杯，獎杯就變得毫無意義。生活最甜美的樂趣，在於承認自己並不特別。」這段文字的涵義，與下列何者最接近？
#####(A)不要強調自己與眾不同
#####(B)追逐名利其實毫無意義
#####(C)認真生活才有甜美的樂趣
#####(D)生活樂在發掘他人的特別
#####答案（Ａ）
###例題（長文）﹕
#####有些科學家提出「獵人與農夫」的理論，認為ADHD者（注意力缺失的過動 兒）其實沒有毛病，只是生錯了時空。現在所謂注意力缺失者的特徵――容易分 心、衝動、冒險性強，其實是遠古打獵採集時生存必要的特徵。當人類進化到農 業社會以後，這些特徵才變得格格不入。也就是說，他們是「獵人」，但是要在 「農夫」的社會裡討生活，往往會被視為異類。 這個理論認為，一萬二千年前，人類走向農業生活開始定居下來後，環境 的改變使得過去的長處變成現在的短處。在遠古時代，如果不眼觀四面，耳聽八方，早就被其他動物吃掉了，不可能成為我們的祖先；如果事情發生不馬上採取 行動，而是三思而後行的話，也只能變成別人的晚餐，活不到成為我們的祖先。而現在學者又把ADHD者叫做有愛迪生基因者，他們的特徵是：思想跳躍、 容易分心、精力旺盛、沒有條理、不耐煩、易衝動、很外向、敢冒險、會發明、 有創造力，而且通常還有領袖魅力。 他們注意力很短暫，但對感興趣的事物可以專注很久，這不是和獵人要一 直不停搜索四周，一旦發現獵物就馬上集中注意力然後追蹤下去相同嗎？他們組 織力不強，沒有條理，很衝動，想到什麼做什麼――獵人不就是看到獵物得馬上拔腿就追嗎？他們沒有時間觀念，不知道做一件事需要花多少時間――這其實是 表示有彈性，一個人的心意如果隨時可變，那就不需要知道時間。他們容易不耐 煩，不能聽從老師的指示――這其實是獵人獨立行事的特性。 上述的理論與觀點尚待可靠證據支持，但如果我們願意換一個角度來看 ADHD者，他們可能並沒有毛病，只是現在教育制度的限制，未必符合他們的天性罷了。

#####34.下列對於ADHD者的敘述，何者最符合作者原意？
#####(A)發明家往往是ADHD者
#####(B)不宜以異類眼光看待ADHD者
#####(C) ADHD者擁有比一般人優秀的基因
#####(D)缺乏打獵機會是現代ADHD者的悲哀



#####35.下列敘述，何者與本文內涵最不相關？
#####(A)強調現代人應培養多元能力，以順應社會趨勢
#####(B)推測ADHD者的特徵可上溯自遠古人類的行為
#####(C)引述學者意見，說明注意力缺失者的行為特徵
#####(D)呼籲教育者應以寬廣的視角引領孩子適性發展


####在與老師進行多次討論之後，我們決定使用google的bert的模型來進行預測，原因是因為NLP太過複雜，依照我們的設備、時間、金費都很難自己從頭訓練一個，所以決定用Bert來接，但由於bert的訓練方式是克漏字選填與預測下一句話，這種詞句鑲嵌的訓練方式，與閱讀測驗不太相似，為此，我們想了兩個方法來進行預測

#方法一：
我們先透過資料整理，將資料用成Bert可接受的形式，然後再用BertForMultipleChoice得出對應之token，之後使用Fine-tuning給他進行後端訓練

#舉例(數字隨便說的)：
####文章：一個幽靈，共產主義的幽靈，在歐洲遊蕩。為了對這個幽靈進行神聖的圍剿，舊歐洲的一切勢力，教皇和沙皇、梅特涅和基佐、法國的激進派和德國的警察，都聯合起來了。
####問題：什麼的幽靈在歐洲遊蕩？
####A：共產主義的幽靈
####B：中華民國的幽靈
#####文章以句號切成兩句 -> [CLS]+句子+[SEP]->所以整個文章轉成：[CLS]一個幽靈，共產主義的幽靈，在歐洲遊蕩。[SEP][CLS]為了對這個幽靈進行神聖的圍剿，舊歐洲的一切勢力，教皇和沙皇、梅特涅和基佐、法國的激進派和德國的警察，都聯合起來了。[SEP]
#####問題+A -> bert -> 一個句向量 -> A向量
#####問題+B -> bert -> 一個句向量 -> B向量
##### [向量1,向量2]與向量A進行餘弦相似度計算 ->得到分數 [0.8,0.4]
##### [向量1,向量2]與向量B進行餘弦相似度計算 ->得到分數 [0.5,0.6]
##### 每個選項選出最大的一個，有就是0.8和0.6，由於0.8>0.6故選A


In [ ]:

pip install transformers

     |████████████████████████████████| 675kB 2.7MB/s 
     |████████████████████████████████| 3.8MB 9.2MB/s 
     |████████████████████████████████| 1.1MB 37.5MB/s 
     |████████████████████████████████| 890kB 40.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=9331fd994a9d5bc7791fe3a294de9d916d43c62fa31d30e4c0e39fd7635b489a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive
%cd python_NCCU


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive
/content/drive/My Drive/python_NCCU


In [ ]:
import pandas as pd
import re
df=pd.read_excel('題目統整.xlsx',header=0)
df=pd.DataFrame(df)


#### Data_國中生國文段考題目

In [ ]:
df.iloc[0,]

科目                                                      國文
文章內容     這三天來都被四面八方傳來的雲雀歌聲引出了門外。這樣明麗的天光,這樣輕快的天樂,誰還坐得住在書...
題目                                     關於這段文句的解析,下列敘述何者正確?
選項(A)                                   運用嗅覺與味覺來摹寫草木與陽光的香氣
選項(B)                                   以水晶、寶玉來比擬周遭景觀的晶瑩明亮
選項(C)                                 透過各種繽紛的色彩描繪田園中的陽光與大地
選項(D)                                 以水晶、佩玉般的清脆聲音描摹陽光下的溪流
題目類型                                                  文章解析
答案                                                       B
文章長度                                                     短
Name: 0, dtype: object

##### 資料整理
* 要把資料轉成可給Bert訓練的格式。文章先以標點符號切句，並把NA去除。
* 閱讀測驗文章格式：[CLS]+句子+[SEP]
* 問題選項格式：[CLS]+題目+[SEP]+選項

In [ ]:
def dot_change(x):
  try:
    X=x.replace(',','，')
    return X
  except:
      return x


In [ ]:
for i in range(len(df.columns)):
  df.iloc[:,i]=df.iloc[:,i].map(dot_change)
#逗號清除

In [ ]:
df.dropna(axis=0,inplace=True)
#清除NA

In [ ]:
df.reset_index(inplace=True)

In [ ]:
def text_change(X):
  y="[CLS]" + X + "[SEP]"
  return y
def text_change2(x):
  text=re.split('[。,！,？,?]',x)
  text2=list(map(text_change,text))
  return text2

In [ ]:
df['文章內容']=df['文章內容'].map(text_change2)


In [ ]:
##將處理好的資料轉為string，以便之後預測使用
a=df['文章內容'].astype(str)
a

0      ['[CLS]這三天來都被四面八方傳來的雲雀歌聲引出了門外[SEP]', '[CLS]這樣明...
1      ['[CLS]寫作題材何處尋[SEP]', '[CLS]其實，你我周遭就有許多溫潤動人的故事...
2      ['[CLS]有一天，父親嚴肅地對我說:『山林是布農男人奔馳和吶喊的地方，除了用腳印標示及保...
3      ['[CLS]鮪魚喜歡較溫暖的海水，常利用洋流洄游[SEP]', '[CLS]鮪魚的鮪魚喜歡...
4      ['[CLS]極偏僻又極佳美的小地方，往往沒有指南，尤其處於不甚有名的國家[SEP]', '...
                             ...                        
880    ['[CLS]「這些童年吃冰的記憶，如今多已消失殆盡，這一代的小孩再也無從體會那種樂趣[SE...
881    ['[CLS]在我星期四的繪畫班中，有位學生每次上課總要帶許多玉蘭花分給同學，所以一到星期四...
882    ['[CLS]在我星期四的繪畫班中，有位學生每次上課總要帶許多玉蘭花分給同學，所以一到星期四...
883    ['[CLS]現代人吃冰是理所當然的一件事，不過古人沒有冰箱怎麼吃冰呢[SEP]', '[C...
884    ['[CLS]現代人吃冰是理所當然的一件事，不過古人沒有冰箱怎麼吃冰呢[SEP]', '[C...
Name: 文章內容, Length: 885, dtype: object

#####將文章內容、題目及選項合併

In [ ]:
##將文章內容、題目及選項合併
def question_choice(content,question,choice):
  question_choice="[CLS]"+content+'[SEP]'+question+'[SEP]'+choice+"[SEP]"
  #print('%d'%(question_choice) , end='\r')
  return question_choice

In [ ]:
df['A']=question_choice(a,df['題目'],df['選項(A)'])
df['B']=question_choice(a,df['題目'],df['選項(B)'])
df['C']=question_choice(a,df['題目'],df['選項(C)'])
df['D']=question_choice(a,df['題目'],df['選項(D)'])


#####進行預測


In [ ]:
##將資料轉為list，以便接下來進行預測
c1=list(df['A'])
c2=list(df['B'])
c3=list(df['C'])
c4=list(df['D'])
c1[0:1]

["[CLS]['[CLS]這三天來都被四面八方傳來的雲雀歌聲引出了門外[SEP]', '[CLS]這樣明麗的天光，這樣輕快的天樂，誰還坐得住在書桌前看書呢[SEP]', '[CLS]站在麻黃樹下，向東看山、看森林、看荒原，向西看田園[SEP]', '[CLS]不論看哪一邊，都是一色澄藍的天展開著，真有這樣不可思議的天色、陽光、大地[SEP]', '[CLS]除非是一種特殊的水晶或什麼寶玉，怎可能鑄造成這樣晶瑩發亮的奇境[SEP]', '[CLS]連空氣都是一種輕質的水晶做的[SEP]', '[CLS]這裡的任一樣東西，只要輕微的敲擊一下，就會發出清脆的聲，無怪四處是雲雀佩玉般的歌音[SEP]', '[CLS][SEP]'][SEP]關於這段文句的解析，下列敘述何者正確?[SEP]運用嗅覺與味覺來摹寫草木與陽光的香氣[SEP]"]

In [ ]:
choices = [c1,c2,c3,c4]

In [ ]:
from transformers import BertTokenizer, BertForMultipleChoice
import torch
import numpy as np


# 取得此預訓練模型所使用的 tokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForMultipleChoice.from_pretrained('bert-base-chinese')
model.eval()


ans=[]
choices = [c1,c2,c3,c4]
choices_1=[]
for i in range(len(c1)):##len=資料長度(以c1數量為準)
  try:
    choices_1.append([c1[i],c2[i],c3[i],c4[i]])
    choices_2=choices_1[i]
    tokens=[tokenizer.encode(s, add_special_tokens=True) for s in choices_2]
    n=min([len(tokens[0]),len(tokens[1]),len(tokens[2]),len(tokens[3])])
    for j in range(4):
      tokens[j]=tokens[j][:n]
    input_ids = torch.tensor(tokens).unsqueeze(0)
    labels = torch.tensor(1).unsqueeze(0)
    outputs = model(input_ids, labels=labels)
    Scores=[]
    Options=['A','B','C','D']
    loss, classification_scores = outputs[:2]
    k=np.array(classification_scores.detach().numpy()).reshape(4,1)
    n2=list(k).index(max(list(k)))
    Options[n2]
    answer=Options[n2]
  except:
    answer=0
  ans.append(answer)




Token indices sequence length is longer than the specified maximum sequence length for this model (623 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (625 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (626 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (628 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (630 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [ ]:
ans

['A',
 'A',
 'A',
 'C',
 'B',
 'D',
 'B',
 0,
 0,
 0,
 0,
 'C',
 'D',
 'D',
 'C',
 'B',
 'C',
 'B',
 'C',
 0,
 0,
 0,
 0,
 0,
 'A',
 'A',
 'B',
 'B',
 'C',
 'B',
 'A',
 'B',
 0,
 0,
 0,
 'A',
 'A',
 'B',
 'D',
 'A',
 'D',
 0,
 0,
 0,
 0,
 0,
 0,
 'D',
 'C',
 'D',
 'A',
 'D',
 'D',
 'C',
 0,
 0,
 0,
 0,
 0,
 0,
 'D',
 'B',
 'B',
 'A',
 'D',
 'C',
 'B',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'B',
 'A',
 'B',
 'C',
 'A',
 'A',
 'D',
 'C',
 'A',
 0,
 'C',
 'B',
 'C',
 0,
 0,
 'A',
 'C',
 'D',
 'A',
 'B',
 'A',
 'A',
 0,
 'B',
 'B',
 'D',
 'C',
 'B',
 'C',
 'C',
 'B',
 'A',
 'C',
 'C',
 'C',
 'A',
 'A',
 0,
 0,
 0,
 0,
 'D',
 'A',
 'D',
 'C',
 'C',
 'D',
 'D',
 'D',
 0,
 0,
 'A',
 'A',
 'C',
 'C',
 'B',
 'A',
 'D',
 'C',
 'B',
 'D',
 'C',
 0,
 0,
 0,
 'D',
 'A',
 'D',
 'A',
 'A',
 'A',
 'A',
 'B',
 'D',
 'C',
 0,
 0,
 0,
 'D',
 'C',
 'D',
 'A',
 0,
 0,
 'B',
 'B',
 'B',
 'C',
 'A',
 'C',
 'D',
 'B',
 'A',
 'B',
 0,
 0,
 0,
 'B',
 'D',
 'D',
 'D',
 'B',
 0,
 0,
 0,
 'A',
 'C',
 'A',
 'B',
 0,
 0,
 0,


In [ ]:
df['Answer']=ans

##### 查看正確率

In [ ]:
##總體正確率
result=df[df['Answer']!=0]
total=pd.value_counts(result['答案']==result['Answer'])
print('正確率為：',total[1]/sum(total)*100)

### 後來發現Bert在輸入資料字數上似乎有所限制，故有些字數過多的題目他會選擇跳過，不進行預測

正確率為： 17.35668789808917


In [ ]:
#文章理解
result_1=result[result['題目類型'].isin(['內容判斷','文章解析','選出對的'])]
total=pd.value_counts(result_1['答案']==result_1['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 14.84375


In [ ]:
#文章解析
result_2=result[result['題目類型'].isin(['比對詞句','詞句理解','詞意理解'])]
total=pd.value_counts(result_2['答案']==result_2['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 14.814814814814813


In [ ]:
#選出錯的
result_3=result[result['題目類型']=='選出錯的']
total=pd.value_counts(result_3['答案']==result_3['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 20.930232558139537


### Data_國中會考題

In [ ]:
df1=pd.read_excel('會考 訓練題庫.xlsx',header=0)
df1=pd.DataFrame(df1)


#### 資料整理


In [ ]:
def dot_change(x):
  try:
    X=x.replace(',','，')
    return X
  except:
      return x


In [ ]:
for i in range(len(df1.columns)):
  df1.iloc[:,i]=df1.iloc[:,i].map(dot_change)
#逗號清除

In [ ]:
df1.dropna(axis=0,inplace=True)
#清除NA

In [ ]:
df1.reset_index(inplace=True)

In [ ]:
def text_change(X):
  y="[CLS]" + X + "[SEP]"
  return y
def text_change2(x):
  text=re.split('[。,！,？,?]',x)
  text2=list(map(text_change,text))
  return text2

In [ ]:
df1['文章內容']=df1['文章內容'].map(text_change2)


In [ ]:
b=df1['文章內容'].astype(str)
b

0      ['[CLS]「彼此無所求的朋友，才可能是真正的朋友[SEP]', '[CLS]」[SEP]']
1     ['[CLS]「常常我們只記得結果的不完美，卻忘記收穫滿滿的過程[SEP]', '[CLS]...
2     ['[CLS]「漢朝刺史巡視的要點有六：一看地方豪強是否恃強凌弱，二看郡守、公侯是否 損公肥...
3     ['[CLS]巴提族人群居在巴基斯坦北部最險惡的高山河谷，六百年前來自西藏[SEP]', '...
4     ['[CLS]宣紙有生熟之別，互有利弊，各隨所好而已[SEP]', '[CLS]生紙易滲水墨...
                            ...                        
59    ['[CLS]三十年前，一個年輕人離開故鄉，開始創造自己的前途[SEP]', '[CLS]少...
60    ['[CLS]我是很晚才使用電腦的那種今之古人[SEP]', '[CLS]四十歲以前，我總覺...
61    ['[CLS]我是很晚才使用電腦的那種今之古人[SEP]', '[CLS]四十歲以前，我總覺...
62    ['[CLS]雪山山脈主脊，從大霸尖山到雪山主峰之間，是一段長達 15 公里，充滿挑 戰的長...
63    ['[CLS]雪山山脈主脊，從大霸尖山到雪山主峰之間，是一段長達 15 公里，充滿挑 戰的長...
Name: 文章內容, Length: 64, dtype: object

####將文章內容、題目及選項合併

In [ ]:
def question_choice(content,question,choice):
  question_choice="[CLS]"+content+'[SEP]'+question+'[SEP]'+choice+"[SEP]"
  #print('%d'%(question_choice) , end='\r')
  return question_choice

In [ ]:
df1['A']=question_choice(b,df1['題目'],df1['選項(A)'])
df1['B']=question_choice(b,df1['題目'],df1['選項(B)'])
df1['C']=question_choice(b,df1['題目'],df1['選項(C)'])
df1['D']=question_choice(b,df1['題目'],df1['選項(D)'])


####進行預測




In [ ]:
##將資料轉為list，以便接下來進行預測
d1=list(df1['A'])
d2=list(df1['B'])
d3=list(df1['C'])
d4=list(df1['D'])
d1[0:1]

["[CLS]['[CLS]「彼此無所求的朋友，才可能是真正的朋友[SEP]', '[CLS]」[SEP]'][SEP]這句話 的涵義與下列何者最接近？[SEP](A)交往若帶有目的，就不是真正的友誼[SEP]"]

In [ ]:
# 取得此預訓練模型所使用的 tokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForMultipleChoice.from_pretrained('bert-base-chinese')
model.eval()


ans1=[]
choices = [d1,d2,d3,d4]
choices_1=[]
for i in range(len(d1)):##len=資料長度(以d1數量為準)
  try:
    choices_1.append([d1[i],d2[i],d3[i],d4[i]])
    choices_2=choices_1[i]
    tokens=[tokenizer.encode(s, add_special_tokens=True) for s in choices_2]
    n=min([len(tokens[0]),len(tokens[1]),len(tokens[2]),len(tokens[3])])
    for j in range(4):
      tokens[j]=tokens[j][:n]
    input_ids = torch.tensor(tokens).unsqueeze(0)
    labels = torch.tensor(1).unsqueeze(0)
    outputs = model(input_ids, labels=labels)
    Options=['A','B','C','D']
    loss, classification_scores = outputs[:2]
    k=np.array(classification_scores.detach().numpy()).reshape(4,1)
    n2=list(k).index(max(list(k)))
    Options[n2]
    answer=Options[n2]
  except:
    answer=0
  ans1.append(answer)




Token indices sequence length is longer than the specified maximum sequence length for this model (846 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (846 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (845 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (845 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (512 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [ ]:
ans1

['D',
 'B',
 'B',
 'A',
 'A',
 'A',
 'A',
 0,
 'D',
 'B',
 'A',
 'B',
 'B',
 'C',
 'B',
 'C',
 'B',
 'D',
 'A',
 'A',
 'C',
 0,
 0,
 0,
 'A',
 'B',
 'A',
 'C',
 'D',
 'A',
 'D',
 'D',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'A',
 'D',
 'B',
 'A',
 'A',
 'A',
 'C',
 'D',
 'A',
 0,
 0,
 0,
 0,
 'B',
 'B',
 'B',
 'C',
 'D',
 'C',
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
df1['Answer']=ans1

#### 查看正確率


In [ ]:
##總體正確率
result=df1[df1['Answer']!=0]
total=pd.value_counts(result['答案']==result['Answer'])
print('正確率為：',total[1]/sum(total)*100)


正確率為： 32.55813953488372


In [ ]:
#文章理解
result_1=result[result['題目類型']=='文章理解']
total=pd.value_counts(result_1['答案']==result_1['Answer'])
print('正確率為：',total[1]/sum(total)*100)


正確率為： 33.33333333333333


In [ ]:
#文章解析
result_2=result[result['題目類型']=='文章解析']
total=pd.value_counts(result_2['答案']==result_2['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 35.294117647058826


In [ ]:
#選出錯的
result_3=result[result['題目類型']=='選出錯的']
total=pd.value_counts(result_3['答案']==result_3['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 50.0


In [ ]:
#詞意理解
result_4=result[result['題目類型'].isin(['詞意理解'])]
total=pd.value_counts(result_4['答案']==result_4['Answer'])
print('正確率為：',total[1]/sum(total)*100)

KeyError: ignored

In [ ]:
total
### 由於此類型題目預測都錯誤，故他的total[1]，也就是true的欄位是0，因此產生上面error的狀況

False    6
dtype: int64

由於準確度有點太低...我們將用第二種方法來做預測

#方法二
####肇因於前一個方法我們使用Fine-tuning給他進行後端訓練，結果並不是太好(比亂猜還糟糕)，所以我們覺得用第二種方法，不進行後端訓練，直接讓Bert pretrained來做。格式與前一個方法類似，首先我們打算把閱讀測驗文章分成一句一句後，丟進Bert做鑲嵌訊練，得到他的詞向量後。在把問題與選項連在一起後，也丟去給Bert做鑲嵌訓練(每個選項分開做一次)，也取他的詞向量。對於文章與ABCD的詞向量進行餘弦相似度測驗後，看與文章中每句的相似度，【選出每個選項最高的分數，比較最高的就是答案】

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 會考 訓練題庫.xlsx to 會考 訓練題庫.xlsx


In [ ]:
import numpy as np
import pandas as pd
import re
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [ ]:
!pip install pytorch_pretrained_bert

In [ ]:
df=pd.read_excel('題目統整.xlsx',header=0)

## 資料整理
#####要把資料轉成可給Bert訓練的格式。文章先以標點符號切句，必把NA去除。
#####閱讀測驗文章格式：[CLS]+句子+[SEP]
#####問題選項格式：[CLS]+題目+[SEP]+選項+[SEP]
#####問題格式：[CLS]+題目+[SEP]

In [ ]:
df.iloc[0,]

科目                            國文
文章內容       「彼此無所求的朋友，才可能是真正的朋友。」
題目              這句話 的涵義與下列何者最接近？
選項(A)        (A)交往若帶有目的，就不是真正的友誼
選項(B)       (B)在朋友困難時伸出援手，才能得到回報
選項(C)      (C)想獲得朋友，須自己先成為值得結交的人
選項(D)    (D)人生有許多事不能獨自完成，可知友誼的重要
題目類型                        詞意理解
答案                             A
文章長度                           短
Name: 0, dtype: object

In [ ]:
df.head()

## 資料整理

In [ ]:
def dot_change(x):
  try:
    X=x.replace(',','，')
    return x
  except:
      return x


In [ ]:
for i in range(len(df.columns)):
  df.iloc[:,i]=df.iloc[:,i].map(dot_change)
#逗號清除

In [ ]:
df.dropna(axis=0,inplace=True)
#清除NA

In [ ]:
df.reset_index(inplace=True)

In [ ]:
def question_choice(question,choice):
  question_choice="[CLS]"+question+'[SEP]'+choice+"[SEP]"
  return question_choice

In [ ]:
df['A']=question_choice(df['題目'],df['選項(A)'])
df['B']=question_choice(df['題目'],df['選項(B)'])
df['C']=question_choice(df['題目'],df['選項(C)'])
df['D']=question_choice(df['題目'],df['選項(D)'])

In [ ]:
def text_change(X):
  y="[CLS]" + X + "[SEP]"
  return y
def text_change2(x):
  text=re.split('[。,！,？,?]',x)
  text2=list(map(text_change,text))
  return text2

In [ ]:
df['文章內容']=df['文章內容'].map(text_change2)

In [ ]:
#製作題目格式
Q=[]
for i in range(len(df)):
  q=text_change(df['題目'][i])
  Q.append(q)
df['Q']=Q

In [ ]:
df.head()

# 開始訓練

## 先用各校段考、基測等題目共1000題做訓練


In [ ]:
test_data=df

In [ ]:
ls

 sample_data/  '會考 訓練題庫.xlsx'   測試.csv


In [ ]:
test_data.reset_index(inplace=True)

In [ ]:
#開啟一個bert model，並直接啟用預測模式
PRETRAINED_MODEL_NAME = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
model = BertModel.from_pretrained(PRETRAINED_MODEL_NAME)
model.eval()
torch.no_grad()

100%|██████████| 382072689/382072689 [00:06<00:00, 55343279.00B/s]


In [ ]:
#將每句閱讀測驗文章轉變成向量
def create_sentence_vector1(text):
  #句子先轉成Tocken
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    #用來分句，但文章每次基本上就一句
    segments_ids = [0] * len(tokenized_text)
    #轉成向量轉並訓練
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
     #Load pre-trained model (weights)
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
    #取隱藏層的最後一層，為我們要的向量(786維)
    sentence_embedding = torch.mean(encoded_layers[11], 1)
    #轉成numpy可用格式
    sentence_embedding=sentence_embedding.detach().numpy()
    sentence_embedding=np.array(sentence_embedding).reshape(1,-1)
    return sentence_embedding

In [ ]:
#將題目加選項變成像向量，基本上跟上面一樣，但是因為有兩句話，所以句數向量那邊有進行調整
def create_sentence_vector2(text):
  try:
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    n=tokenized_text.index('[SEP]')
    segments_tensors = torch.tensor([0] * n + [1] * (len(tokenized_text)-n),dtype=torch.long)
    tokens_tensor = torch.tensor([indexed_tokens])
    # Load pre-trained model (weights)
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
    sentence_embedding = torch.mean(encoded_layers[11], 1)
    #轉成numpy可用格式
    sentence_embedding=sentence_embedding.detach().numpy()
    sentence_embedding=np.array(sentence_embedding).reshape(1,-1)
    return sentence_embedding
  except:
    pass

In [ ]:
#問題轉向量
test_data['Q']=test_data['Q'].map(create_sentence_vector1)

In [ ]:
for i in range(len(test_data['文章內容'])):
  test_data['文章內容'][i]=list(map(create_sentence_vector1,test_data['文章內容'][i]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
test_data['A']=test_data['A'].map(create_sentence_vector2)
test_data['B']=test_data['B'].map(create_sentence_vector2)
test_data['C']=test_data['C'].map(create_sentence_vector2)
test_data['D']=test_data['D'].map(create_sentence_vector2)

In [ ]:
test_data.to_csv('向量.csv')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#方案一
#舉例(數字隨便說的)：
####文章：一個幽靈，共產主義的幽靈，在歐洲遊蕩。為了對這個幽靈進行神聖的圍剿，舊歐洲的一切勢力，教皇和沙皇、梅特涅和基佐、法國的激進派和德國的警察，都聯合起來了。
####問題：什麼的幽靈在歐洲遊蕩？
####A：共產主義的幽靈
####B：中華民國的幽靈
#####文章以句號切成兩句 -> bert -> 轉成2句向量(向量1、向量2)
#####問題+A -> bert -> 一個句向量 -> A向量
#####問題+B -> bert -> 一個句向量 -> B向量
##### [向量1,向量2]與向量A進行餘弦相似度計算 ->得到分數 [0.8,0.4]
##### [向量1,向量2]與向量B進行餘弦相似度計算 ->得到分數 [0.5,0.6]
##### 每個選項選出最大的一個，有就是0.8和0.6，由於0.8>0.6故選A


In [ ]:
Answer=[]
for i in range(len(test_data)):
  try:
    scoresA=np.array(0)
    scoresB=np.array(0)
    scoresC=np.array(0)
    scoresD=np.array(0)
    t=np.array(test_data['文章內容'][i])[:,0]
    scoresA=np.sort(cosine_similarity(t,np.array(test_data['A'][i])).reshape(1,len(t)))
    scoresB=np.sort(cosine_similarity(t,np.array(test_data['B'][i])).reshape(1,len(t)))
    scoresC=np.sort(cosine_similarity(t,np.array(test_data['C'][i])).reshape(1,len(t)))
    scoresD=np.sort(cosine_similarity(t,np.array(test_data['D'][i])).reshape(1,len(t)))
    Scores=[]
    Options=['A','B','C','D']
    Scores.append(scoresA[:,-1:].sum())
    Scores.append(scoresB[:,-1:].sum())
    Scores.append(scoresC[:,-1:].sum())
    Scores.append(scoresD[:,-1:].sum())
    answer_index=Scores.index(max(Scores))
    answer=Options[answer_index]
  except:
    answer=0
  Answer.append(answer)

In [ ]:
test_data['Answer']=Answer

In [ ]:
final=test_data[test_data['Answer']!=0]
total=pd.value_counts(final['答案']==final['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 28.227848101265824


In [ ]:
final3=final[final['題目類型'].isin(['內容判斷','文章解析','選出對的'])]
total=pd.value_counts(final3['答案']==final3['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 26.005361930294907


In [ ]:
final4=final[final['題目類型'].isin(['比對詞句','詞句理解','詞意理解'])]
total=pd.value_counts(final4['答案']==final4['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 36.84210526315789


In [ ]:
final5=final[final['題目類型'].isin(['選出錯的'])]
total=pd.value_counts(final4['答案']==final4['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 36.84210526315789


In [ ]:
len(final4)

52

#方案二
###嘗試看看，先用一樣的餘弦相似度，找出最相似的句子，再單純依單一句子與問題+選想進行餘弦相似度計算，選出最大值為答案
#舉例(數字隨便說的)：
####文章：一個幽靈，共產主義的幽靈，在歐洲遊蕩。為了對這個幽靈進行神聖的圍剿，舊歐洲的一切勢力，教皇和沙皇、梅特涅和基佐、法國的激進派和德國的警察，都聯合起來了。
####問題：什麼的幽靈在歐洲遊蕩？
####A：共產主義的幽靈
####B：中華民國的幽靈
#####文章以句號切成兩句 -> bert -> 轉成2句向量(向量1、向量2)
#####文章->bert(向量3)
#####問題+A -> bert -> 一個句向量 -> A向量
#####問題+B -> bert -> 一個句向量 -> B向量
##### [向量1,向量2]與向量3進行餘弦相似度計算 ->得到分數 [0.9,0.5]
##### 因0.9>0.5故選擇第一句(向量1)當作最相關向量
##### [向量1]與向量A進行餘弦相似度計算 ->得到分數 [0.8]
##### [向量1]與向量B進行餘弦相似度計算 ->得到分數 [0.6]
##### 由於0.8>0.6故選A為正確答案


In [ ]:
#選出最相關的句子
most_related_sentence=[]
n3=len(test_data)
Answer=[]
for i in range(n3):
  t1=np.array(test_data['文章內容'][i])[:,0]
  related=cosine_similarity(t1,np.array(test_data['Q'][i]))
  related_sort=np.sort(related.reshape(1,len(t1)))
  related_list=list(related.reshape(1,len(t1))[0,:])
  nn=related_list.index(related_sort[0,-1:])
  try:
    scoresA=np.array(0)
    scoresB=np.array(0)
    scoresC=np.array(0)
    scoresD=np.array(0)
    t=t1[nn,:].reshape(1,768)
    scoresA=np.sort(cosine_similarity(t,np.array(test_data['A'][i])).reshape(1,len(t)))
    scoresB=np.sort(cosine_similarity(t,np.array(test_data['B'][i])).reshape(1,len(t)))
    scoresC=np.sort(cosine_similarity(t,np.array(test_data['C'][i])).reshape(1,len(t)))
    scoresD=np.sort(cosine_similarity(t,np.array(test_data['D'][i])).reshape(1,len(t)))
    Scores=[]
    Options=['A','B','C','D']
    Scores.append(scoresA[:,-1:].sum())
    Scores.append(scoresB[:,-1:].sum())
    Scores.append(scoresC[:,-1:].sum())
    Scores.append(scoresD[:,-1:].sum())
    answer_index=Scores.index(max(Scores))
    answer=Options[answer_index]
  except:
    answer=0
  Answer.append(answer)

In [ ]:
test_data['Answer']=Answer

In [ ]:
final=test_data[test_data['Answer']!=0]
total=pd.value_counts(final['答案']==final['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 26.83544303797468


### 比較兩個方案之後，發現方案一正確率較高，故選擇方案一來對會考題目進行預測

In [ ]:
big_test=pd.read_excel('會考 訓練題庫.xlsx')

In [ ]:
for i in range(len(big_test.columns)):
  big_test .iloc[:,i]=big_test.iloc[:,i].map(dot_change)
#逗號清除
big_test.dropna(axis=0,inplace=True)
#清除NA
big_test.reset_index(inplace=True)
big_test['A']=question_choice(big_test['題目'],big_test['選項(A)'])
big_test['B']=question_choice(big_test['題目'],big_test['選項(B)'])
big_test['C']=question_choice(big_test['題目'],big_test['選項(C)'])
big_test['D']=question_choice(big_test['題目'],big_test['選項(D)'])
big_test['文章內容']=big_test['文章內容'].map(text_change2)
#製作題目格式
Q=[]
for i in range(len(big_test)):
  q=text_change(big_test['題目'][i])
  Q.append(q)
big_test['Q']=Q

In [ ]:
for i in range(len(big_test['文章內容'])):
  big_test['文章內容'][i]=list(map(create_sentence_vector1,big_test['文章內容'][i]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
big_test['A']=big_test['A'].map(create_sentence_vector2)
big_test['B']=big_test['B'].map(create_sentence_vector2)
big_test['C']=big_test['C'].map(create_sentence_vector2)
big_test['D']=big_test['D'].map(create_sentence_vector2)

In [ ]:
Answer=[]
for i in range(len(big_test)):
  try:
    scoresA=np.array(0)
    scoresB=np.array(0)
    scoresC=np.array(0)
    scoresD=np.array(0)
    t=np.array(big_test['文章內容'][i])[:,0]
    scoresA=np.sort(cosine_similarity(t,np.array(big_test['A'][i])).reshape(1,len(t)))
    scoresB=np.sort(cosine_similarity(t,np.array(big_test['B'][i])).reshape(1,len(t)))
    scoresC=np.sort(cosine_similarity(t,np.array(big_test['C'][i])).reshape(1,len(t)))
    scoresD=np.sort(cosine_similarity(t,np.array(big_test['D'][i])).reshape(1,len(t)))
    Scores=[]
    Options=['A','B','C','D']
    Scores.append(scoresA[:,-1:].sum())
    Scores.append(scoresB[:,-1:].sum())
    Scores.append(scoresC[:,-1:].sum())
    Scores.append(scoresD[:,-1:].sum())
    answer_index=Scores.index(max(Scores))
    answer=Options[answer_index]
  except:
    answer=0
  Answer.append(answer)

In [ ]:
big_test['Answer']=Answer
final=big_test[big_test['Answer']!=0]
total=pd.value_counts(final['答案']==final['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 38.095238095238095


In [ ]:
final['題目類型'].unique()

array(['詞意理解', '文章解析', '選出錯的', '文章理解'], dtype=object)

### 答對率：
#### Bert:38%
#### 社會二修齊：73%
#### 應數二嘉恆：85%
#### 社會二懷沁：85%
#### 社會二嘉緯：81%

###查看一下各種題型的正確率，題型我們分成四類，分為'詞意理解'、'文章解析'、'選出錯的'、'文章理解' 四大分類

In [ ]:
big_test[big_test['題目類型']=='文章理解']['題目'].head()

12       下列敘述何者最符合這段文字的觀點？
18        這句話的意旨，與下列何者最接近？
20       下列敘述何者最符合這段文字的內容？
21    根據本文，藥廠生產藥物的原則為下列何者？
25     文中「最貴重的東西」最可能是下列何者？
Name: 題目, dtype: object

In [ ]:
#文章理解(即理解文章主旨)
final2=final[final['題目類型'].isin(['文章理解'])]
total=pd.value_counts(final2['答案']==final2['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 11.11111111111111


In [ ]:
big_test[big_test['題目類型']=='文章解析']['題目'].head()

2          根據這段文 字，漢朝刺史的職責最可能是下列何者？
3    根據這段文句，下列何者最可能是巴提族和雪巴族 都具有的特色？
5                  根據這段文字，下列敘述何者正確？
6        根據這段文字，下列何者最符合作者對《史記》 的看法？
7         依本文所舉古今文人對小孩的好惡，下列敘述何者正確？
Name: 題目, dtype: object

In [ ]:
#文章解析(須看懂題目後，根據題目敘述選出與選項最近者)
final3=final[final['題目類型'].isin(['文章解析'])]
total=pd.value_counts(final3['答案']==final3['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 50.0


In [ ]:
big_test[big_test['題目類型']=='詞意理解']['題目'].head()

0       這句話 的涵義與下列何者最接近？
1        這句話的涵義與下列何者最接近？
13     這段文字的涵義與下列 何者最接近？
24     這句話的涵義， 與下列何者最接近？
54    這 段文字的涵義，與下列何者最接近？
Name: 題目, dtype: object

In [ ]:
#詞意理解(選出選項之語句與題目之語句最相近者)
final4=final[final['題目類型'].isin(['詞意理解'])]
total=pd.value_counts(final4['答案']==final4['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 57.14285714285714


In [ ]:
big_test[big_test['題目類型']=='選出錯的']['題目'].head()

4                         根據這段文字，下列推論何者錯誤？
10    根據本文，關於《徐霞客遊記》與其他晚明文人遊記的比較，下列敘述何者錯誤？
11                          根據本文，下列敘述何者錯誤？
15                     這段文字所要說明的內容不包含下列何者？
17                       這句話的 涵義與下列何者相去最遠？
Name: 題目, dtype: object

In [ ]:
#選出錯的(題目敘述題及「下列何者錯誤」者)
final5=final[final['題目類型'].isin(['選出錯的'])]
total=pd.value_counts(final5['答案']==final5['Answer'])
print('正確率為：',total[1]/sum(total)*100)

正確率為： 42.857142857142854


# 結論
1.    Bert在做會考的題目時，成果明顯好過一般的選擇題
2.  然而總正確率依舊大概只有40%，其中文章理解的題目明顯低於其他，文章解析與詞意理解答得最好
3.給bert做下游訓練反而更糟糕

#分析

1.總體正確率的話，可能肇因於國中會考題目太過於困難，有太多隱喻的敘述，加上背後有很多意識形態，即使是我們自己寫大概也只對70、80%。所以以符號學來說的話，題目描述時常不是只停留在語言系統層面，導致機器難以理解

2.此外，中文閱讀測驗的答案並不一定在文章之中，因為我們的方法是看每一句的相似度給分數，所以當問題需要先備知識或日常生活中的經驗依據時，便可能有問題。

3.但這並不能說明我們的考試背後沒有邏輯，Bert的訓練方式是詞句的鑲嵌，與寫考卷不同，也就是說原本的訓練出來的形式邏輯就不同。如果說人對於社會的詮釋是基於我們理解世界的方式與經驗，那AI理解的便是由演算法(方式)與資料(經驗)所構成。而Bert被訓練的方式與資料原本便不是為了考試，且它缺乏人類有意識的面對社會的這種反身性思考，對於方式無法自我修正，所以對於考試的失力可能在與此

4.但很有趣的一點是，對於文章的解析答對率大於對文章的理解。我覺得因為Bert經由維基百科資料的經驗，可能可以藉由那些資料判斷對於關鍵字的解析，且解析這件事隱含著的主觀意義較少，選出錯的、詞句理解(這種甚至可以說答案就在文章中)亦同。但如果是理解的，理論上每個人對於文章都有自己的解釋，但國文考試要求我們去選出那個他們所希冀的答案，這代表機器對於一段文字的理解，其實跟人類是有十分大的差距的。

5.機器讀了蘇軾也不代表會有靈魂
